# Tarea 4. Modelos de Lenguaje Estadísticos

Guillermo Segura Gómez

## Modelos de Lenguaje y Evaluación
**1. Preprocese todos los tuits de agresividad (positivos y negativos) según su intuición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, etc.). Agregue tokens especiales de <s> y </s> según usted considere (e.g., al inicio y final de cada tuit). Defina su vocabulario y enmascare con <unk> toda palabra que no esté en su vocabulario.**

### Modelos de Lenguaje

Un modelo de lenguaje típicamente asigna probabilidades a secuencias de palabras o genera secuencias de palabras basándose en las probabilidades aprendidas. Estas probabilidades reflejan qué tan probable es que una palabra siga a otra u otras en una secuencia. Los modelos de lenguaje pueden ser de varios tipos, incluidos los basados en reglas gramaticales, los estadísticos, y más recientemente, los basados en redes neuronales, como los modelos de lenguaje transformadores.

### Modelos de Lenguaje de N-gramas
Los modelos de lenguaje de n-gramas son un tipo específico de modelo de lenguaje estadístico que se basa en la premisa de que la probabilidad de una palabra en una secuencia depende solo de las $n-1$ palabras anteriores. Un "n-grama" es, por lo tanto, una secuencia de $n$ elementos (por ejemplo, palabras) del texto. Por ejemplo:

- Un 1-grama (o unigrama) considera cada palabra de forma aislada.
- Un 2-grama (o bigrama) considera pares de palabras consecutivas.
- Un 3-grama (o trigrama) considera tríos de palabras consecutivas, y así sucesivamente.

En un modelo de n-gramas, la probabilidad de una palabra dada está condicionada por las $n-1$ palabras anteriores. Estos modelos son relativamente simples de construir y pueden ser muy efectivos para ciertas tareas, especialmente cuando los recursos computacionales son limitados o cuando se dispone de una cantidad limitada de datos de entrenamiento. Sin embargo, los modelos de n-gramas tienen limitaciones, como la dificultad para manejar contextos más largos y el problema de la "explosión combinatoria" de posibles n-gramas a medida que $n$ aumenta, lo que puede hacer que el modelo sea menos práctico para valores grandes de $n$.

Para poder construir un modelo de n-gramas primero necesitamos pre procesar nuestro corpus para que sea mas sencillo de manejar. Por ejemplo, podemos trabajar únicamente palabras en minúscula. Además, es necesario tener delimitadores de oraciones \<s\> y \</s\> y definir un vocabulario. Cualquier palabra fuera del vocabulario le asignamos el token \<unk\>. Vamos a trabajar con los tweets de agresividad del corpus [MEX-A3T](https://sites.google.com/view/mex-a3t/home?authuser=0) con el que hemos estado trabajando.

In [1]:
# Función que extrae el texto de dos archivos. Uno el de los documentos, otro el de las etiquetas
def get_text_from_file(path_corpus, path_truth):

    tr_text = []
    tr_labels = []

    with open(path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_text += [tweet]
        for label in f_truth:
            tr_labels += [label]

    return tr_text, tr_labels

In [2]:
path_text = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_train.txt"
path_labels = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_train_labels.txt"

path_text_val = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_val.txt"
path_labels_val = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_val_labels.txt"

tr_text, tr_labels = get_text_from_file(path_text, path_labels) # Importamos los datos de entrenamiento
val_text, val_labels = get_text_from_file(path_text_val, path_labels_val) # Importamos los datos de test o validación

Lo primero que tenemos que hacer es delimitar nuestro vocabulario. Limitamos solo las palabras en minúsculas. 
Además debemos agregar los tokens de inicio y final a cada documento del corpus.

In [19]:
import nltk
from nltk.tokenize import TweetTokenizer
import re

In [20]:
# Función para preprocesar los textos
def preprocess_texts(texts):
    # Convertir a minúsculas y eliminar números y caracteres especiales
    texts_lower = [re.sub(r"http\S+|[^a-zA-Z\s]", "", doc.lower()) for doc in texts]
    return texts_lower

# Función que agrega delimitadores de oraciones a un corpus
def addlimiters(texts):
    texts_limites = []
    for doc in texts:
        newText = "<s>" + doc + "</s>"
        texts_limites.append(newText)

    return texts_limites

# Preprocesar el texto
tr_text_min = preprocess_texts(tr_text)

# Agregar delimitadores de oración
tr_text_limited = addlimiters(tr_text_min)


Ahora tokenizamos el texto.

In [21]:
tokenizer = TweetTokenizer() # Inicializar tokenizer
corpus_palabras = []

for doc in tr_text_limited:
    corpus_palabras += tokenizer.tokenize(doc)

fdist = nltk.FreqDist(corpus_palabras)

print(f"El tamaño del corpus es:", len(corpus_palabras))
print(f"El tamaño del vocabulario es:", len(fdist))

El tamaño del corpus es: 94261
El tamaño del vocabulario es: 12388


Definimos una función para construir el vocabulario. Utilizamos una longitud del vocabulario de cinco mil. 

In [26]:
# Función para ordenar las frecuencias
def  SortFrecuency(freqdist):
    # List comprenhension
    aux = [(freqdist[key], key) for key in freqdist]
    aux.sort() # Ordena la lista
    aux.reverse() # Cambiar el orden

    return aux

# Ordenamos y obtenemos el vocabulario
voc = SortFrecuency(fdist)
voc = voc[:5000]
voc[:5]

[(5278, '<s>'), (5278, '</s>'), (3102, 'que'), (3095, 'de'), (2268, 'la')]

Ahora reemplazamos las palabras desconocidas con el token \<unk\>

In [27]:
# Función para reemplazar palabras fuera del vocabulario con <unk>
def replace_unknowns(tokenized_texts, vocab):
    # Reemplazar palabras que no están en el vocabulario con <unk>
    return [[word if word in vocab else '<unk>' for word in doc] for doc in tokenized_texts]

tr_text_final = replace_unknowns(corpus_palabras, voc)

**2. Entrene tres modelos de lenguaje sobre todos los tuits: $P_{unigramas}(w_1^n)$, $P_{bigramas}(w_1^n)$, $P_{trigramas}(w_1^n)$. Para cada uno proporcione una interfaz (función) sencilla para  $P_{n-grama}(w_1^n)$ y  $P_{n-grama}(w_1^n | w_{n_N+1}^{n-1})$.  Los modelos deben tener una estrategia común para lidiar con secuencias no vistas. Puede optar por un suavizamiento Laplace o un Good-Turing discounting. Muestre un par de ejemplos de como funciona, al menos uno con una palabra fuera del vocabulario.**

El modelo de n-gramas que vamos a construir es una función que recibe una palabra $w_ y asigna la probabilidad en base a las $n-1$ palabras de las que este formada la cadena. Para ejemplificar esto, consideremos el ejemplo mostrado en el capitulo 3 del libro de Daniel Jurafsky [1].

En el capitulo 3 del libro se busca encontrar un modelo que calcule la probabilidad de que la siguiente palabra a la oración *“
its water is so transparent that* sea *the* es decir

$$
P(\text{the}|\text{its water is so transparent that})
$$

* En un modelo de unigramas se asume que la aparición de cada palabra es independiente de las palabras antes o después de ella en la secuencia. Esto significa que no se considera el contexto en el que aparece la palabra; cada palabra es tratada de forma aislada.

    La probabilidad de una secuencia de palabras $ w_1^n $ en un modelo de unigramas se calcula simplemente como el producto de las probabilidades individuales de cada palabra en la secuencia:

    $$ P_{\text{unigramas}}(w_1^n) = P(w_1) \times P(w_2) \times \ldots \times P(w_n) = \prod_{i=1}^{n} P(w_i) $$

    Donde:
    - $ P_{\text{unigramas}}(w_1^n) $ es la probabilidad de la secuencia de palabras $ w_1, w_2, \ldots, w_n $ bajo el modelo de unigramas.
    - $ P(w_i) $ es la probabilidad de la palabra individual $ w_i $.

    La probabilidad $P(w_i)$ de cada palabra individual se estima generalmente a partir del corpus de entrenamiento. La forma más simple de estimar $P(w_i)$ es usar la frecuencia relativa de la palabra en el corpus:

    $$ P(w_i) = \frac{\text{Frecuencia de } w_i}{\text{Número total de palabras en el corpus}} $$

* En el modelo de bigramas, calculamos la probabilidad según $n-1$ palabras, es decir

    $$
    P(\text{the}|\text{that})
    $$

    La suposición de que la probabilidad de una palabra depende solo en la palabra previa se conoce como **suposición de Markov**. Para calcular la probabilidad en un modelo de bigramas de una palabra $w_n$ dada una previa palabra $w_{n-1}$ calculamos la cuenta de cuentas ocasiones aparecen las palabras juntas $C(w_{n-1} w_n)$ y la normalizamos por la suma de todos los bigramas que comparten la misma palabra $w_{n-1}$

    $$
    P(w_n|w_{n-1}) = \frac{C(w_{n-1} w_n)}{ \sum_w C(w_{n-1})}
    $$

    Podemos simplificar esta ecuación, ya que la suma de todas las cuentas de los bigramas que empiezan con la misma palabra $w_{n-1}$, debe de ser igual a la cuenta de los unigramas para esta palabra $w_{n-1}$, entonces 

    $$
    P(w_n|w_{n-1}) = \frac{C(w_{n-1} w_n)}{ C(w_{n-1})}
    $$

    Para una secuencia de tres palabras \(w_1, w_2, w_3\), la probabilidad bajo el modelo de bigramas sería:

    $$ P_{\text{bigramas}}(w_1, w_2, w_3) = P(w_1) \times P(w_2 | w_1) \times P(w_3 | w_2) $$

In [34]:
tweets_trannig = ['I am Sam\n', 'Sam I am\n', ' I do not like green eggs and ham\n']


'I am Sam\n'

**3. Construya un modelo interpolado con valores $\lambda$ fijos**
$$
\hat{P}\left(w_n \mid w_{n-2} w_{n-1}\right)=\lambda_1 P\left(w_n \mid w_{n-2} w_{n-1}\right)+\lambda_2 P\left(w_n \mid w_{n-1}\right)+\lambda_3 P\left(w_n\right)
$$

**Para ello experimente con el modelo en particiones estratificadas de $80 \%, 10 \%$ y $10 \%$ para entrenar (train), ajuste de parámetros ( $v a l$ ) y prueba (test) respectivamente. Muestre como bajan o suben las perplejidades en validación, finalmente pruebe una vez en test. Para esto puede explorar algunos valores $\vec{\lambda}$ y elija el mejor. Pruebe las siguientes: $[1 / 3,1 / 3,1 / 3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1]$ y $[.1, .4, .5]$.**

## Generación de Texto

Para esta parte reentrenará su modelo de lenguaje interpolado para aprender los valores $\lambda$:
$$
\hat{P}\left(w_n \mid w_{n-2} w_{n-1}\right)=\lambda_1 P\left(w_n \mid w_{n-2} w_{n-1}\right)+\lambda_2 P\left(w_n \mid w_{n-1}\right)+\lambda_3 P\left(w_n\right)
$$

Realice las siguientes actividades:

**1.Proponga una estrategia con base en Expectation Maximization (investigue por su cuenta sobre EM) para encontrar buenos valores de interpolación en $\hat{P}$ usando todo el dataset de agresividad (Se adjunta un material de apoyo). Para ello experimente con el modelo en particiones estratificadas de 80\%, 10\% y 10\% para entrenar (train), ajustar parámetros (val) y probar (test) respectivamente. 1 Muestre como bajan las perplejidades en 5 iteraciones que usted elija (de todas las que sean necesarias de acuerdo a su EM) en validación, y pruebe una vez en test. Sino logra hacer este punto, haga los siguientes dos con el modelo de lenguaje con algunos $\lambda$ fijos.**


## Referencias

[1] Keselj, Vlado. "Book Review: Speech and Language Processing by Daniel Jurafsky and James H. Martin." Computational Linguistics 35.3 (2009).